# Olymploration: Who wins the most medals at the Olympics?

Rooted in rich history, symbolism and values associated with the Olympic games, Olympic medals are considered highly prestigious, often representing the pinnacle of athletic achievement. Winning an Olympic medal is a recognition of excellence on the international stage, and it often holds cultural and national significance. It is an opportunity for athletes from all nations, big or small, to transcend national boundaries and compete together. This tradition began in 1896 in Athens, Greece and with the exception of the games not being held due to the two world wars, the Summer games have been held every 4 years. The first Winter Olympics were held in Chamonix, France, in 1924. The goal of this project is to predict which country will win the most medals at the Olympic games. 

In [ ]:
#import packages needed

import os
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import glob2 as glob
import missingno as msno
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import r2_score
from sklearn import preprocessing 

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
#read in files
relative_path = "data/raw/*.csv" 
file_list = glob.glob(relative_path)

dataframes = []


if not file_list:
    print("No CSV files found in the specified path.")
else:
    file_list.sort() #to stop the order from changing

    # Loop through the sorted file list and read each CSV file into a df
    for file in file_list:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading file {file}: {e}")

In [ ]:
 #name the dfs
df_athletes = dataframes[0]
df_event_results = dataframes[1]
df_medals = dataframes[2]
df_results = dataframes[3]
df_country = dataframes[4]
df_games = dataframes[5]
df_gdp_mean = dataframes[6]
df_pop_mean = dataframes[7]

In [ ]:
dataframes = {'df_athletes': df_athletes, 
       'df_country': df_country, 
       'df_event_results': df_event_results, 
       'df_games': df_games, 
       'df_medals': df_medals, 
       'df_results': df_results,
             'df_gdp_mean' : df_gdp_mean,
             'df_pop_mean' : df_pop_mean}

In [ ]:
#check missing values
def check_missing_values(df):
    return df.isnull().sum()


# check_missing_values(df_event_results)
# check_missing_values(df_country)
# check_missing_values(df_games)
# check_missing_values(df_athletes)
# check_missing_values(df_medals)
# check_missing_values(df_results)

def count_na(df):
    def helper(val):
        return val == 'na'
    return df.applymap(helper).sum() / df.shape[0]

# for key, value in dataframes.items():
#     print(f'\n\n{key}\n\n')
#     print(count_na(value))

In [ ]:
# do we have winter olympics available as much? 
dict(Counter([i[5:] for i in df_games['edition'].unique()]))

In [ ]:
df_games[df_games['end_date'].isna()]

#missing values for end_Date, start_date if the games were not held,
#isHeld is NA if the games took place which. These missing values will not affect our analysis

In [ ]:
# how many athletes take part by country
df_event_results.groupby(['edition', 'country_noc'], as_index=False)['athlete'].count().head()

In [ ]:
# one athlete can take part in several events
df_event_results.groupby(['edition', 'country_noc'])['athlete'].unique().reset_index(name='unique_athletes').assign(athlete_count=lambda x: x['unique_athletes'].apply(len)).head()


In [ ]:
# are the dates reasonable? 
#pd.to_datetime(df_games['year'], format='%Y').dt.year.sort_values(ascending=False)

In [ ]:
# inconsistencies between competition_date and start_date/end_date 
#df_games[['start_date', 'end_date', 'competition_date', 'isHeld']]

In [ ]:
#total medals 

df_medals_agg = df_medals.groupby('country')[['gold', 'silver', 'bronze', 'total']].sum()
df_medals_agg.describe()

Historically, USA, the Soviet Union, Germany, Great Britain and France have won the most medals at the Olympics.The Soviet Union won the second most at 1204 followed by Germany at 1098. In terms of gold and silver medals won, USA leads by a mile.  USA should be considered an outlier while modeling.  The table above shows the descriptive statistics for the medals data. As you can see, the lower 75% of countries have historically won a total of only 98 medals which confirms that only a handful of countries dominate at the games.

<strong><i>Who has won the most medals? </i> </strong>

In [ ]:
#historically, who won the most medals?? 

df_medals2 = df_medals.groupby('country_noc')[['total','gold', 'silver', 'bronze']].sum()
df_medals2.reset_index(inplace=True)


top5_rows = df_medals2.nlargest(5, 'total')

top5_countries = top5_rows['country_noc'].tolist()
top5_medals = top5_rows['total'].tolist()

print(f"The top 5 countries are: {top5_countries}")
print(f"The corresponding counts are: {top5_medals}")

In [ ]:
# Melt the DataFrame to long format 
top_50 = df_medals2.nlargest(50, 'total')
df_medals3 = top_50.drop(['total'], axis=1)
df_medals_melted = pd.melt(df_medals3, id_vars='country_noc', var_name='Medal', value_name='Count')

df_medals_melted.head()

In [ ]:
#who won the most medals

df_stacked = df_medals_melted.pivot_table(index='country_noc', columns='Medal', values='Count', fill_value=0)

df_stacked_sorted = df_stacked.sum(axis=1).sort_values(ascending=False).index
df_stacked = df_stacked.loc[df_stacked_sorted]


colors = {'gold': 'gold', 'silver': 'silver', 'bronze': 'brown'}

plt.figure(figsize=(12, 8))
for medal_type, color in colors.items():
    plt.bar(df_stacked.index, df_stacked[medal_type], color=color, label=medal_type)

plt.title('Top 50 Countries by Medal Type: USA dominates')
plt.xlabel('Country')
plt.ylabel('Number of Medals')
plt.legend(title='Medal Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45, ha='right') 
plt.show()

<strong><i> Has anything changed in the past 20 years?</i></strong>

In [ ]:
#has anything changed in the past 20 years?

#filter df to include 2004 Athens games onwards

df_last20 = df_medals[['year', 'country', 'total']].copy()

df_last20 = df_last20.loc[df_last20['year'] >= 1992]
df_last20.head()

In [ ]:
total_by_country = df_last20.groupby('country')['total'].sum()
top_10_countries = total_by_country.sort_values(ascending=False).head(12)
top_10_countries_list = total_by_country.sort_values(ascending=False).head(12).index.tolist()

recent_games  = df_last20[df_last20['country'].isin(top_10_countries_list)]

recent_games.head()

In [ ]:

plt.figure(figsize=(12, 8))
sns.lineplot(x='year', y='total', hue='country', data=recent_games, marker='o', markersize=8)

plt.xticks(recent_games['year'].unique())

plt.title('Total Medals by Country in the past 20 years')
plt.xlabel('Year')
plt.ylabel('Total Medals')
plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')  
plt.xticks(rotation=45, ha='right')  
plt.show()

In the past 30 years, USA continues to remain extremely dominant at the Summer Olympics (first Summer Olympics in the graph took place in 1992 and first Winter Olympics took place in 1994). The Russian Federation/Soviet Union which historically had the second most medals is not as dominant as before. In the past 4 editions of the Summer games, China has won the second most number of medals, bypassing Russia. Great Britain has also steadily increased their medal count in the past 30 years.
However at the Winter Olympics, Norway's athletes have been outpacing the U.S. and Germany.


<strong> <i>Do GDP and Population have an impact on the number of medals won?<i><strong>

In [ ]:
#GDP and population - using the past 10 year avg as a proxy as data is available 1960 onwards for some countries,
#while data for some is available even later
df_country_gdp = pd.read_csv('data/raw/country_gdp.csv')
df_country_gdp.head(10)

In [ ]:
df_country_gdp.drop(['Indicator Name', 'Indicator Code'], axis=1, inplace=True)

In [ ]:
df_country2 = pd.merge(df_country, df_country_gdp, how='left', left_on='country', right_on='Country Name')

exclude_columns = ['country_noc', 'country', 'Country Code', 'Country Name']
numeric_columns = df_country2.select_dtypes(include=['number']).columns

# Filter out columns to exclude from the division
columns_to_divide = numeric_columns.difference(exclude_columns)

# Perform division only on numeric columns
df_country2[columns_to_divide] = df_country2[columns_to_divide].div(1e9)

df_country2.head()


In [ ]:
df_country2.iloc[:, -10:] = df_country2.iloc[:, -10:].apply(pd.to_numeric, errors='coerce')

# # Calculate the mean of the last 10 columns while ignoring NaN values
df_country2['mean_gdp_last_10yr_bn'] = df_country2.iloc[:, -10:].mean(axis=1, skipna=True)

df_country2.head()

In [ ]:
missing_values_gdp = df_country2['mean_gdp_last_10yr_bn'].isnull().sum()
missing_values_gdp

We would like to use GDP and Population  as additional independent variables in our models and examine whether it has an impact on the medals tally or not. One of the limitations we are facing with the World Bank data is that it is available only 1960 onwards with data for some countries being available much later. The data for the past 10 years is available and we plan on using the 10yr mean as a proxy for GDP in our model. An alternative way is to create dummy variables after categorizing the countries based on the range of their population, eg- high, medium, low. The chart below shows the countries with the highest mean GDP over the past 10 years and  we guess that GDP  is positively correlated with the medals count.  

In [ ]:
df_sorted = df_country2.sort_values(by='mean_gdp_last_10yr_bn', ascending=False)
top_10_values = df_sorted.head(10)

plt.barh(top_10_values['country_noc'], top_10_values['mean_gdp_last_10yr_bn'], color='teal')
plt.xlabel('GDP($bn)')
plt.title('Mean GDP over the last 10 years by country (in $bn), USA leads...')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_pop = pd.read_csv('data/raw/country_population.csv')
df_pop.drop(['Country Name',  'Indicator Name', 'Indicator Code'], axis=1, inplace=True)
df_pop.head()

In [ ]:
#df_pop.drop(['Indicator Name', 'Indicator Code'], axis=1, inplace=True)
df_pop2 = pd.merge(df_country, df_pop, how='left', left_on='country', right_on='Country_Updated')
df_pop2.head(3)

In [ ]:
exclude_columns = ['country_noc', 'country', 'Country Code', 'Country_Updated']
df_pop2.loc[:, df_pop2.columns.difference(exclude_columns)] /= 1e6 # convert to millions 

In [ ]:
#following a similar approach of taking the 10 year mean
df_pop2['mean_pop_last_10yr_mm'] = df_pop2.iloc[:, -10:].mean(axis=1)
df_country_pop = df_pop2[['country_noc', 'country', 'Country Code', 'mean_pop_last_10yr_mm']].copy()
#df_country_pop.to_clipboard(index=False)

In [ ]:
missing_values_pop = df_country_pop['mean_pop_last_10yr_mm'].isnull().sum()
missing_values_pop

We followed a similar approach for Population and calculated the 10yr mean as a proxy. Apart from China, USA, Russia, Japan, other countries with the highest mean population generally do not do well at the Games. Having a larger population could also mean less resources to invest in training athletes and could in fact, have a negative impact on medals won.

In [ ]:
df_sorted_pop = df_country_pop.sort_values(by='mean_pop_last_10yr_mm', ascending=False)
top_10_values_pop = df_sorted_pop.head(10)


plt.barh(top_10_values_pop['country_noc'], top_10_values_pop['mean_pop_last_10yr_mm'], color='salmon')
plt.xlabel('Population($MM)')
plt.title('Mean Population over the last 10 years by country')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
#group by total medals for each country

df_grouped = df_medals.groupby('country_noc')['total'].sum()
df_grouped = pd.DataFrame(df_grouped)
df_grouped.reset_index(inplace=True)

In [ ]:
#check correlation between variables and medals tally

df_merged = pd.merge(df_grouped, df_country2[['country_noc','mean_gdp_last_10yr_bn']
                                                                        ],how='left', on='country_noc')

df_merged2 = pd.merge(df_merged, df_country_pop[['country_noc','mean_pop_last_10yr_mm']
                                                                        ],how='left', on='country_noc')


df_merged2.head()


In [ ]:
#exclude China, India, USA
excluded_countries = ['CHN', 'IND', 'USA']
df_filtered = df_merged2[~df_merged2['country_noc'].isin(excluded_countries)]
sns.scatterplot(y='total', x='mean_pop_last_10yr_mm', data=df_filtered, color='salmon')

# Add labels and title
plt.xlabel('Population (in MM)')
plt.ylabel('Medals Won')
plt.title('Population vs Medals won')

# Show the plot
plt.show()

We excluded China and USA along with India which has the second highest 10 yr mean population and it appears that some countries with smaller populations have won more medals. However, there are also countries that are very tiny, have very small populations and hence, have very low participation at the Olympics.

In [ ]:
sns.scatterplot(y='total', x='mean_gdp_last_10yr_bn', data=df_filtered, color='teal')

# Add labels and title
plt.xlabel('GDP (in $bn)')
plt.ylabel('Medals Won')
plt.title('GDP vs Medals won')

# Show the plot
plt.show()

The scatter plot confirms that GDP is positvely correlated with medals one. We still have countries with missing GDP but they are either very small or nations that had a name change, we will be excluding those records.

In [ ]:
# excluded_countries = ['USA']
# df_filtered = df_merged3[~df_merged3['country_noc'].isin(excluded_countries)]

# sns.scatterplot(y='total', x='total_athletes', data=df_filtered, color='mediumpurple')


# # Add labels and title
# plt.xlabel('Contingent Size')
# plt.ylabel('Medals Won')
# plt.title('Contingent Size vs Medals won')

# # Show the plot
# plt.show()

In [ ]:
# join athletes with events to get the sport for each athlete, it is not in df_athletes

df_athletes_sport = pd.merge(df_athletes[['name', 'sex', 'height', 'weight']],
                              df_event_results[['athlete', 'sport']], left_on='name', right_on='athlete', how='left')

df_athletes_sport.head()

In [ ]:
df_athletes_sport = df_athletes_sport.dropna()

In [ ]:
df_athletes_sport['height'] = pd.to_numeric(df_athletes_sport['height'], errors='coerce')  
df_athletes_sport['weight'] = pd.to_numeric(df_athletes_sport['weight'], errors='coerce')  

# Group by 'sport' and calculate the average height and weight
average_data = df_athletes_sport.groupby(['sport', 'sex'])[['height', 'weight']].mean().reset_index()

#select a few sports 

select_sports = ['Archery', 'Artistic Gymnastics', 'Athletics',
                'Badminton', 'Boxing', 'Cross Country Skiing',
                'Cycling Track', 'Diving', 'Equestrian Dressage',
                'Fencing', 'Hockey', 'Judo',
                 'Rowing', 'Shooting', 'Handball',
                 'Swimming', 'Triathlon', 'Volleyball',
                 'Weightlifting', 'Wrestling', 'Basketball', 'Table Tennis', 'Tennis'
                ]

#keep only selected sports
mask = average_data['sport'].isin(select_sports) 
selected_df = average_data[mask]

<strong><i>How does an athlete’s height and weight affect their choice of sport?<strong><i>

In [ ]:
# Sort  in descending order
average_height_by_sport = selected_df.sort_values(by='height', ascending=False)
average_height_by_sport.dropna(inplace=True)

colors = {'Female': 'thistle', 'Male': 'skyblue'}

plt.figure(figsize=(5, 5))
sns.barplot(y='sport', x='height', hue='sex', data=average_height_by_sport, palette=colors)
plt.title('AvHeight by Sport: Basketball players are the tallest')
plt.xlabel('Average Height in cm')
plt.ylabel('')
plt.legend(title='Gender', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.subplots_adjust(right=0.75)
plt.show()

In [ ]:
# Sort weight in descending order
average_weight_by_sport = selected_df.sort_values(by='weight', ascending=False)
average_weight_by_sport.dropna(inplace=True)

colors = {'Female': 'thistle', 'Male': 'skyblue'}

plt.figure(figsize=(5, 5))
sns.barplot(y='sport', x='weight', hue='sex', data=average_weight_by_sport, palette=colors)
plt.title('Average Weight by Sport: Basketball players also weigh the most')
plt.xlabel('Average Weight in kg')
plt.ylabel('')
plt.legend(title='Gender', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.subplots_adjust(right=0.75)
plt.show()

We selected some of the most popular events at the Olympics to see if there is any obvious trend in the height and weight of athletes competing in those sports. The charts above show the average height and weight of all the athletes playing a particular sport, missing records were excluded. Basketball players tend to be the tallest and also weigh the most but the difference in the average height across sports is minimal for both males and females. On the other hand, the difference in weight between males and females across the different sports is more obvious. Furthermore, gymnasts, boxers and divers weigh less than athletes playing basketball or handball.

<strong><i>Which countries do better at team vs individual events?</i></strong>

In [ ]:
#which countries do better at team vs individual events

df_event_results2 = df_event_results[df_event_results['medal'] != 'na']
df_event_results2.head()

In [ ]:
#group by country

df_team = df_event_results2.groupby(['country_noc', 'isTeamSport']).result_id.count().reset_index()
df_team = pd.DataFrame(df_team)
df_team

In [ ]:
#filter by team vs individual
filtered_data = df_team[df_team['isTeamSport'] == False]

top_20_countries = filtered_data.groupby('country_noc')['result_id'].sum().nlargest(20).index

top_20_data = filtered_data[filtered_data['country_noc'].isin(top_20_countries)]

top_20_data = top_20_data.sort_values(by='result_id', ascending=False)

#lollipop chart
plt.figure(figsize=(5, 3))
plt.stem(top_20_data['country_noc'], top_20_data['result_id'], basefmt='k-', use_line_collection=True, markerfmt='purple',
         linefmt='purple')
plt.title('Top 20 Countries winning Individual Events')
plt.xlabel('Country')
plt.ylabel('Events won')
plt.xticks(rotation=45, ha='right')


plt.show()

In [ ]:
filtered_data2 = df_team[df_team['isTeamSport'] == True]

top_20_countries2 = filtered_data2.groupby('country_noc')['result_id'].sum().nlargest(20).index


top_20_data2 = filtered_data2[filtered_data2['country_noc'].isin(top_20_countries2)]

# Sort the data for better visualization
top_20_data2 = top_20_data2.sort_values(by='result_id', ascending=False)

# Create a lollipop chart
plt.figure(figsize=(5,3))
plt.stem(top_20_data2['country_noc'], top_20_data2['result_id'], basefmt='k-', use_line_collection=True, markerfmt='purple',
         linefmt='purple')
plt.title('Top 20 Countries winning Team Events')
plt.xlabel('Country')
plt.ylabel('Events won')
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.show()

While the USA is an outlier doing well at both individual and team events, Great Britain comes next in terms of most medals won at team events. Countries like Canada, Australia, the Netherlands among others also happen to do better at team events than individual events.

<strong><i>Who is the greatest Olympian of all time?<strong><i>

In [ ]:
#greatest Olympian of all time 

greatest_olympian = df_event_results2.groupby(['athlete', 'sport', 'country_noc']).result_id.count().sort_values(ascending=False)
greatest_olympian = pd.DataFrame(greatest_olympian)
greatest_olympian.rename({'result_id':'medals_won'}, axis=1, inplace=True)
greatest_olympian.reset_index(inplace=True)
greatest_olympian.head()

In [ ]:
greatest_olympian.info()

In [ ]:
greatest_olympian2 = greatest_olympian.sort_values(by='medals_won', ascending=False)
greatest_olympian2 = greatest_olympian2.head(20)
plt.figure(figsize=(6, 5))
bars = plt.barh(greatest_olympian2['athlete'], greatest_olympian2['medals_won'], color='darkslateblue')
plt.xlabel('Medals Won')
plt.ylabel('Athlete')
plt.title('Greatest Olympians of All Time by Medals Won')

for bar, label in zip(bars, greatest_olympian2['country_noc']):
    plt.text(bar.get_width() + 0.2, bar.get_y() + bar.get_height() / 2, label, ha='left', va='center', color='black', fontsize=8)

plt.show()

Michael Phelps is well-known as the greatest Olympian of all time but in second place is Larisa Latynina, a gymnast from the former Soviet Union who won 18 medals. 25% of the top 20 are from the USA.

<strong><i>Which gender wins more medals? </strong></i>

In [ ]:
#which gender wins more medals
gender_medals = pd.merge(df_athletes[['name', 'sex']], df_event_results2[['athlete','medal', 'sport','result_id']],
                        left_on="name", right_on="athlete")
gender_medals2 = gender_medals.groupby('sex').athlete.count()
gender_medals2=pd.DataFrame(gender_medals2)
gender_medals2.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(4, 4))
sns.set(style="whitegrid")
plt.pie(gender_medals2['athlete'], labels=gender_medals2.sex, autopct='%1.1f%%', startangle=90,
        colors=['thistle', 'skyblue'])


plt.title('Medal Winners by Gender')

plt.show()


In [ ]:
#gold medal winners

gender_gold = gender_medals[gender_medals['medal'] == 'Gold']

gender_gold2 = gender_gold.groupby('sex').athlete.count()
gender_gold2=pd.DataFrame(gender_gold2)
gender_gold2.reset_index(inplace=True)
gender_gold2

In [ ]:
plt.figure(figsize=(4, 4))
sns.set(style="whitegrid")
plt.pie(gender_gold2['athlete'], labels=gender_gold2.sex, autopct='%1.1f%%', startangle=90,
        colors=['thistle', 'skyblue'])


plt.title('Gold Medal Winners by Gender')

plt.show()


72% of all medal winners are Male, with a similar stat for gold medal winners. This is expected given that the historical participation rate of female athletes  is 26%.

In [ ]:
gender_all = df_athletes.groupby('sex')['name'].count()
gender_all

In [ ]:
# how many athletes take part by country
df_event_results.groupby(['edition', 'country_noc'], as_index=False)['athlete'].count().head()

In [ ]:
# one athlete can take part in several events
df_event_results.groupby(['edition', 'country_noc'])['athlete'].unique().reset_index(name='unique_athletes').assign(athlete_count=lambda x: x['unique_athletes'].apply(len)).head()


Merge: Join medals and games

In [ ]:
# df_medals['edition_id'].nunique()
# df_games['edition_id'].nunique()


# drop war and future events
df_games = df_games.drop(df_games[~df_games['edition_id'].isin(df_medals['edition_id'])].index)


df_games_medals = pd.merge(df_medals, pd.merge(df_games, df_country, on='country_noc', how='left'), on=['edition_id', 'edition', 'year'], how='left',
        suffixes=('_performing', '_host'))
df_games_medals.head()

In [ ]:
#special or one time events
df_games_medals[(df_games_medals['edition_id'] == 4)|(df_games_medals['edition_id'] == 48)]

# drop the one-time events
df_games_medals = df_games_medals.drop(df_games_medals[(df_games_medals['edition_id'] == 4)|
                                                       (df_games_medals['edition_id'] == 48)].index)

In [ ]:
df_games_medals.describe()

In [ ]:
#some more eda

def eda_visualization(df):
    numerical_columns = df.select_dtypes(include='number').columns
    categorical_columns = df.select_dtypes(exclude='number').columns

    for column in numerical_columns:
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        sns.histplot(df[column], kde=True)
        plt.title(f'Distribution of {column}')
        
        plt.subplot(1, 2, 2)
        ax = sns.boxplot(data=df, x=column, showfliers=False)
        plt.title(f'Boxplot of {column} Without Outliers')
        
        def add_quartile_labels(ax):
            for line in ax.lines:
                x, y = line.get_xydata()[0]  
                ax.text(x, y, f"{x:.0f}", ha='left', va='center')
        
        add_quartile_labels(ax)

        plt.tight_layout()
        plt.show()

    for column in categorical_columns:
        plt.figure(figsize=(8, 4))
        categories_to_show = df[column].value_counts().index[:30]

        if not categories_to_show.empty:
            sns.countplot(data=df[df[column].isin(categories_to_show)], x=column, order=categories_to_show)

            for idx, count in enumerate(df[df[column].isin(categories_to_show)][column].value_counts()):
                plt.text(idx, count + 1, str(count), ha='center', va='bottom', fontsize=8)

            plt.title(f'Count of each category in {column}')
            plt.xticks(rotation=90, ha='right', fontsize=7)
            plt.tight_layout()
            plt.show()
        else:
            print(f"No categories to show for column {column}.")

In [ ]:
eda_visualization(df_games_medals)

In [ ]:
#further analyze df_games_medals 
df_comparison = df_games_medals.groupby(['country_host', 'edition', 'year', 'country_performing'], as_index=False)[['total', 'gold', 'silver', 'bronze']].sum() \
    .groupby('edition', group_keys=False) \
    .apply(lambda x: x.nlargest(8,'total'))

In [ ]:
df_comparison.head()

In [ ]:
#add dummy variabe for season
df_comparison['season'] = [1 if 'Winter' in edition else 0 for edition in df_comparison['edition']]

In [ ]:
#add another dummy variable for is_host

df_comparison['is_host'] = np.where(df_comparison['country_host'] == df_comparison['country_performing'], 1, 0)

In [ ]:
df_summer = df_comparison[df_comparison['season'] == 0]
df_summer2 = df_summer.groupby(['year', 'is_host'])['total'].sum()
df_summer2 = pd.DataFrame(df_summer2)
df_summer2.reset_index(inplace=True)
df_summer2.head()

<strong><i>Does being a host country appear to furnish an advantage?</i></strong>
<br>
For most years, the host country wins medals. The number of medals won by the host country appears to be more sizable during the summer games than the winter ones. Though some countries are repeatedly hosting the games and could simply be strong contenders, it is likely still an influential factor. We will include is_host and season as two binary variables to capture this potential effect. 


In [ ]:
#plot summer games
plt.figure(figsize=(12, 8))
sns.barplot(x='total', y='year', hue='is_host', data=df_summer2, dodge=False, ci=None,palette={1: 'mediumvioletred', 0: 'lightseagreen'}, orient='h')
plt.title('Total Medal Count by Year')
plt.xlabel('Total Medals')
plt.ylabel('Year')
plt.show()

In [ ]:
#plot winter games

df_winter = df_comparison[df_comparison['season'] == 1]
df_winter2 = df_winter.groupby(['year', 'is_host'])['total'].sum()
df_winter2 = pd.DataFrame(df_winter2)
df_winter2.reset_index(inplace=True)
df_winter2.head()

In [ ]:

plt.figure(figsize=(12, 8))
sns.barplot(x='total', y='year', hue='is_host', data=df_winter2, dodge=False, ci=None,palette={1: 'cornflowerblue', 0: 'peachpuff'}, orient='h')
plt.title('Total Medal Count by Year')
plt.xlabel('Total Medals')
plt.ylabel('Year')
plt.show()

In [ ]:
#lets explore the summer games by excluding usa 

df_summer_nousa = df_comparison[(df_comparison['season'] == 0) & (df_comparison['country_performing'] != 'United States')]
df_summer_nousa2 = df_summer_nousa.groupby(['year', 'is_host'])['total'].sum()
df_summer_nousa2= pd.DataFrame(df_summer_nousa2)
df_summer_nousa2.reset_index(inplace=True)
df_summer_nousa2.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x='total', y='year', hue='is_host', data=df_summer_nousa2, dodge=False, ci=None,palette={1: 'mediumvioletred', 0: 'lightseagreen'}, orient='h')
plt.title('Total Medal Count by Year')
plt.xlabel('Total Medals')
plt.ylabel('Year')
plt.show()

<strong> Start building model:</strong>

In [ ]:
def plot_residuals(actual_values, predicted_values, title='Residual Plot'):  
    sns.set_style('darkgrid')
    residuals = actual_values - predicted_values

    plt.figure(figsize=(5, 3))
    sns.scatterplot(x=actual_values, y=residuals, color='lightblue')
    plt.axhline(y=0, color='red', linestyle='--')

    plt.title(f'{title}', fontsize=12)
    plt.xlabel('Actual Values', fontsize=10)
    plt.ylabel('Residuals', fontsize=10)
    plt.grid(True)
    plt.show()

In [ ]:
results = []

In [ ]:
def record_results(model):
    global results
    results.append({
        'model':f'{model}',
        'MSE train': f'{mse_train}',
        'MSE test': f'{mse_test}',
        'RMSE train': f'{np.sqrt(mse_train)}',
        'RMSE test': f'{np.sqrt(mse_test)}',
        'R2 train': f'{r2_score(y_train, base_train_predictions)}',
        'R2 test': f'{r2_score(y_test, base_predictions)}'
    })
    print('results recorded')

In [ ]:
#Building a baseline model

#Model 1: Using data for the 2020 Olympics only

#start building df to predict medal count at the 2024 summer games

#feel free to change any of this, remove or add other variables.etc., haven't scaled or anything, this was just to test

df_games_medals['is_host'] = np.where(df_games_medals['country_host'] == df_games_medals['country_performing'], 1, 0)

#add gdp and population-using already processed df saved in the data folder

df_gdp_pop = pd.merge(df_gdp_mean,df_pop_mean, on='country_noc', how='left')
df_gdp_pop.drop(['country_x', 'Country Code_x', 'country_y', 'Country Code_y'], axis=1, inplace=True)

In [ ]:
#cols for model
df_cols = df_games_medals[['year','edition', 'country_noc_performing', 'gold', 'silver', 'bronze','total', 'is_host']]

In [ ]:
df_cols.rename({'country_noc_performing':'country_noc'}, axis=1, inplace=True)
df_cols.head()

In [ ]:
#merge
df_model = pd.merge(df_cols, df_gdp_pop[['country_noc','mean_gdp_last_10yr_bn', 'mean_pop_last_10yr_mm']], on='country_noc', how='left')


In [ ]:
#add contingent size 
contingent_size = df_event_results.groupby(['edition', 'country_noc'], as_index=False)['athlete'].nunique()

df_model2 = pd.merge(df_model, contingent_size[['edition','country_noc','athlete']], on=['edition','country_noc'], how='left')
df_model2.head()

In [ ]:
df_model2.rename({'athlete':'contingent_size'}, axis=1,inplace=True)

In [ ]:
#number of events they participate in

event_participation =  df_event_results.groupby(['edition', 'country_noc'], 
                                                                 as_index=False)['event'].nunique()

df_model3 = pd.merge(df_model2, event_participation[['edition','country_noc','event']], on=['edition','country_noc'], how='left')
df_model3.rename({'event':'event_count'}, axis=1, inplace=True)
df_model3.head()

In [ ]:
#has country won medals in the past

prior_medals = pd.DataFrame(df_medals.groupby('country_noc')['total'].sum())
prior_medals.reset_index(inplace=True)
prior_medals.head()

In [ ]:
#add dummy
prior_medals['prior_medals'] = (prior_medals['total'] > 0).astype(int)

In [ ]:
#using 2020 data only
df_2020 = df_model3.loc[df_model3['year']==2020]

In [ ]:
df_2020.drop(['year', 'edition', 'gold', 'silver', 'bronze'], axis=1, inplace=True)

In [ ]:
#add medals in prior games 

df_medals_2016= df_medals.loc[df_medals['year']==2016]
df_medals_2016.rename({'total':'2016_medals'}, axis=1, inplace=True)

In [ ]:
#merge
df_2020_base = pd.merge(df_2020, df_medals_2016[['country_noc', '2016_medals']], how ='left', on=['country_noc'])
df_2020_base.head()

In [ ]:
df_2020_base['2016_medals'].fillna(0, inplace=True)

In [ ]:
df_2020_base.dropna(inplace=True)

In [ ]:
df_2020_base.rename({'total':'total_medals'}, axis=1, inplace=True)

In [ ]:
df_2020_base = pd.merge(df_2020_base, prior_medals[['country_noc', 'prior_medals']], 
                        on='country_noc', how='left')

In [ ]:
#run a basic linear reg model
X = df_2020_base[['is_host', 'mean_gdp_last_10yr_bn', 'mean_pop_last_10yr_mm', 'contingent_size',
            'event_count', '2016_medals', 'prior_medals']]
y = df_2020_base['total_medals']

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.25, random_state=101) 

In [ ]:
base_model = LinearRegression()
base_model.fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

In [ ]:
mse_test = mean_squared_error(y_test, base_predictions)
print(mse_test)

In [ ]:
base_train_predictions = base_model.predict(X_train)
mse_train = mean_squared_error(y_train, base_train_predictions)
print(mse_train)

In [ ]:
#add predictions to df 
df_2020_base['predicted_total_medals'] = base_model.predict(X)
df_2020_base.head()

In [ ]:
coefficients = base_model.coef_

# coefficients with their corresponding feature names
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})

print(coef_df)

In [ ]:
plot_residuals(y_test, base_predictions, title='Residual Plot Base 2020 LR Model')

In [ ]:
record_results(model='Base 2020 LR')

In [ ]:
#Model 2- exlcuding USA, just testing if there is any improvement


df_2020_wo_usa = df_2020_base.loc[df_2020_base['country_noc']!= 'USA']

X = df_2020_wo_usa[['is_host', 'mean_gdp_last_10yr_bn', 'mean_pop_last_10yr_mm', 'contingent_size',
            'event_count', '2016_medals', 'prior_medals']]
y = df_2020_wo_usa['total_medals']

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.25, random_state=101) 

In [ ]:
base_model = LinearRegression()
base_model.fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

In [ ]:
mse_test = mean_squared_error(y_test, base_predictions)
print(mse_test)

In [ ]:
base_train_predictions = base_model.predict(X_train)
mse_train = mean_squared_error(y_train, base_train_predictions)
print(mse_train)

In [ ]:
plot_residuals(y_test, base_predictions, title='Residual Plot Base 2020 LR Model')

In [ ]:
record_results(model='Base 2020 LR no USA')

In [ ]:
coefficients = base_model.coef_

# coefficients with their corresponding feature names
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})

print(coef_df)

In [ ]:
# Model 2: Trying with all the years instead of just 2020

In [ ]:
# adding the season dummy 
df_model3['winter'] = [1 if 'Winter' in edition else 0 for edition in df_model3['edition']]

In [ ]:
# merging 
df_model3 = df_model3.merge(prior_medals[['country_noc', 'prior_medals']], on='country_noc')

In [ ]:
previous_year_df = df_model3.groupby(['country_noc', 'year', 'winter'], as_index=False)['total'].sum()

In [ ]:
# we need this dataframe to add the previous year's(event's) medal total to the current year's
previous_year_df['year'] = previous_year_df['year']+4

In [ ]:
previous_year_df.rename(columns={'total':'previous_year_medals'}, inplace=True)

In [ ]:
df_model3 = df_model3.merge(previous_year_df, on=['country_noc', 'year', 'winter'], how='left')

In [ ]:
df_model3['previous_year_medals'].fillna(0, inplace=True)

In [ ]:
df_model3.drop(['edition', 'gold', 'silver', 'bronze'], axis=1, inplace=True)

In [ ]:
# we will switch to a data source that has these missing values in the future and deletion wont be needed
df_model3.dropna(inplace=True)

In [ ]:
X = df_model3[['year','is_host', 'mean_gdp_last_10yr_bn', 'mean_pop_last_10yr_mm', 'contingent_size',
            'event_count', 'previous_year_medals', 'prior_medals',
              'winter']]

y = df_model3['total']

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.25, random_state=101, stratify=df_model3['year']) 

In [ ]:
base_model.fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

In [ ]:
mse_test = mean_squared_error(y_test, base_predictions)
print(np.sqrt(mse_test), mse_test) 

In [ ]:
base_train_predictions = base_model.predict(X_train)
mse_train = mean_squared_error(y_train, base_train_predictions)
print(np.sqrt(mse_train), mse_train) 

In [ ]:
coefficients = base_model.coef_

# coefficients with their corresponding feature names
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})

coef_df

In [ ]:
plot_residuals(y_test, base_predictions, title='Residual Plot Test All Years Model')
plot_residuals(y_train, base_train_predictions)

In [ ]:
record_results(model='Base all years LR')

In [ ]:
pd.DataFrame(results).T

<strong> Evaluation </strong>

In [ ]:
# modify and add however u like!! not convinced everything is correct either

The base LR model using 2020 data performs best in terms of MSE, especially when dropping the US as a country. The magnitude of the errors is lower, and the number of medals predicted go from being off by 6.5 to 6.3 (decimals are a result of using a linear regression model on count data). However, the difference between the train and test MSEs in the second fitting that drops the USA is larger. Since such a significant outlier was removed, the model is perhaps overfitting on noise that it can now better perceive. Some collinearity between variables like contingent size and event count, prior medals and previous year medals, etc. could also contribute. The R2 for the fitting without the USA is lower than that of the first, possibly because removing a row of data in a small dataset can cause information loss that affects the overall goodness of fit. Meanwhile, the base model applied to all years performs the worst, with a larger difference in train-test scores and generally worse metrics. 

Considering that the assumptions of linear regression are violated (e.g. heteroscedasticity, independence of observations, possible collinearity, etc.), this base model performs well. Going forward, it will be important to address the time component of the data (whether it means focusing on a smaller dataset or using all years) and the treatment of outliers like the USA (e.g., taking the log may improve the model). Moreover, since the model performs well initially, the possibilty of information leakage exists; the model may be capable of inferring a certain year's medal count based on the medal counts available for that year in the training data. 

Data size is another aspect: if only one year is used, the data shrinks to around 80 rows, which is not a robust number. We may need to explore data generation/oversampling techniques, or find a time-series appropriate solution that allows the use of more years. In terms of modelling, we look forward to exploring methods for count data, such as random forest, gradient boosting, poisson regression, etc. 

This base model furnishes a first glimpse of the feature importances. Being a host increases the medal tally by 1.5, all things equal, while winter seems to decrease it by 1.1. Previous year medals and contingent size also have a somewhat smaller, positive contribution. Applied to 2020 data, the important features become the number of events partaken in. Interestingly, the sign switches for 2016_medals after removing the USA; they are decreasing the the medals won by a country in 2020. We are currently working on adding more variables, for example related to climate, pollution, distance to the host country, etc. These are country inherent characteristics that are likely to impact the performance of athletes and in turn, the total medals. 

In [ ]:
# add pollution variable (greenhouse gas emissions) 
ghg_df = pd.read_excel('data/raw/GHG.xlsx', sheet_name='GHG_per_capita_by_country')

In [ ]:
# ghg_df.isna().sum() no missing values 

In [ ]:
ghg_df.drop(columns='Country', inplace=True)

In [ ]:
ghg_df = pd.melt(ghg_df, id_vars=['EDGAR Country Code'], var_name='year', value_name='ghg_per_capita')

In [ ]:
ghg_df['year'] = ghg_df['year'].astype(int)

In [ ]:
# KOS = SERBIA?

In [ ]:
country_code_mappings = {'GRC':'GRE',
 'DEU':'GER',
 'DNK':'DEN',
 'CHE':'SUI',
 'NLD':'NED',
 'ZAF':'RSA',
 'URY':'URU',
 'PRT':'POR',
 'CHL':'CHI',
 'PHL':'PHI',
 'LVA':'LAT',
 'LKA':'SRI',
 'IRN':'IRI',
 'PRI':'PUR',
 'BGR':'BUL',
 'NGA':'NGR',
 'MNG':'MGL',
 'NER':'NIG',
 'BMU':'BER',
 'ZWE':'ZIM',
 'DZA':'ALG',
 'ZMB':'ZAM',
 'CRI':'CRC',
 'IDN':'INA',
 'HRV':'CRO',
 'SVN':'SLO',
 'MYS':'MAS',
 'TON':'TGA',
 'SAU':'KSA',
 'VNM':'VIE',
 'BRB':'BAR',
 'KWT':'KUW',
 'ARE':'UAE',
 'PRY':'PAR',
 'SCG':'SRB',
 'WSM':'SAM',
 'SDN':'SUD',
 'MUS':'MRI',
 'TGO':'TOG',
 'GRD':'GRN',
 'BWA':'BOT',
 'GTM':'GUA',
 'SCG':'MNE',
 'FJI':'FIJ',
 'ITA':'SMR',
 'BFA':'BUR'}

In [ ]:
ghg_df.replace(country_code_mappings, inplace=True)

In [ ]:
# just using this to get the country_noc > country map to compare with the one in the greenhouse gas data ones
df_model3.merge(df_country, on='country_noc', how='left').groupby(['country_noc', 'country'], as_index=False).count()

In [ ]:
# getting unmatched country ids even though theyre in both datasets ahh
df_model3 = df_model3.merge(ghg_df, how='left', left_on=['country_noc', 'year'], right_on=['EDGAR Country Code', 'year'])

In [ ]:
df_model3[(df_model3['year']>1970)&df_model3.isna().any(axis=1)]['country_noc'].unique()

In [ ]:
ghg_df['EDGAR Country Code'].unique()